<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CEIA/NLP/Desafio3/Canciones_rock_nacional.txt', sep ='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,"Estoy muy solo y triste, acá,"
1,en este mundo abandonado.
2,"Tengo la idea, la de irme"
3,al lugar que yo más quiera.
4,"Me falta algo para ir,"


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1795


### 1 - Preprocesamiento

In [5]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [6]:
# Demos un vistazo
sentence_tokens[:2]

[['estoy', 'muy', 'solo', 'y', 'triste', 'acá'],
 ['en', 'este', 'mundo', 'abandonado']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [8]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=2,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1795


In [11]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 888


### 3 - Entrenar el modelo generador

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=10000,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Se han truncado las últimas 5000 líneas del flujo de salida.
Loss after epoch 5000: 7172.0
Loss after epoch 5001: 6936.0
Loss after epoch 5002: 7012.0
Loss after epoch 5003: 7088.0
Loss after epoch 5004: 6976.0
Loss after epoch 5005: 7088.0
Loss after epoch 5006: 6408.0
Loss after epoch 5007: 7244.0
Loss after epoch 5008: 6928.0
Loss after epoch 5009: 7140.0
Loss after epoch 5010: 7376.0
Loss after epoch 5011: 6556.0
Loss after epoch 5012: 7080.0
Loss after epoch 5013: 6968.0
Loss after epoch 5014: 6856.0
Loss after epoch 5015: 7504.0
Loss after epoch 5016: 6996.0
Loss after epoch 5017: 6836.0
Loss after epoch 5018: 7184.0
Loss after epoch 5019: 6748.0
Loss after epoch 5020: 6860.0
Loss after epoch 5021: 6852.0
Loss after epoch 5022: 6592.0
Loss after epoch 5023: 6876.0
Loss after epoch 5024: 7068.0
Loss after epoch 5025: 6736.0
Loss after epoch 5026: 6944.0
Loss after epoch 5027: 7380.0
Loss after epoch 5028: 6804.0
Loss after epoch 5029: 6800.0
Loss after epoch 5030: 7212.0
Loss afte

(59629985, 97830000)

### 4 - Ensayar

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["jugo"], topn=10)

[('tomate', 0.5903346538543701),
 ('gobierno', 0.5345038771629333),
 ('cantores', 0.47404277324676514),
 ('frío', 0.4719996452331543),
 ('habitación', 0.4637477397918701),
 ('difícil', 0.4549567997455597),
 ('melancolía', 0.45000892877578735),
 ('perra', 0.4487699270248413),
 ('habitual', 0.44686758518218994),
 ('guitarra', 0.44616591930389404)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["extraño"], topn=10)

[('rey', 0.693453311920166),
 ('largo', 0.6371826529502869),
 ('espía', 0.5736585259437561),
 ('pelo', 0.553762674331665),
 ('espectador', 0.5094844698905945),
 ('monstruo', 0.4764305353164673),
 ('aprendió', 0.4660499095916748),
 ("brotha'", 0.46100127696990967),
 ('pirata', 0.4530787169933319),
 ('hijo', 0.4469160735607147)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rasguña"], topn=10)

[('paredes', 0.5695474147796631),
 ('dicen', 0.5071654319763184),
 ('venas', 0.5033513307571411),
 ('manos', 0.49420464038848877),
 ('vagando', 0.4905485510826111),
 ('piedras', 0.47827088832855225),
 ('e', 0.47531968355178833),
 ('raíces', 0.46418964862823486),
 ('¡al', 0.46369510889053345),
 ('gato', 0.46294450759887695)]

In [35]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["bestia"], topn=10)

[('pop', 0.6918991208076477),
 ('nuestro', 0.6530320644378662),
 ('fasola', 0.6000924110412598),
 ('gran', 0.5714876651763916),
 ('amigo', 0.5704290866851807),
 ('vigilia', 0.5443476438522339),
 ('eso', 0.5220789313316345),
 ('vía', 0.5123901963233948),
 ('juan', 0.504176914691925),
 ('llorar', 0.48909252882003784)]

In [39]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["alegría"], topn=10)

[('u', 0.6534020304679871),
 ('dale', 0.537522554397583),
 ('uuh', 0.5033060312271118),
 ('sigo', 0.45541414618492126),
 ('ofrecer', 0.44626104831695557),
 ('look', 0.42969048023223877),
 ('natural', 0.4292832016944885),
 ('vengo', 0.41554126143455505),
 ('brillar', 0.41515886783599854),
 ('importa', 0.41093316674232483)]

In [34]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["flaca"], topn=10)

[('une', 0.20636089146137238),
 ('pido', 0.17407289147377014),
 ('creo', 0.16527391970157623),
 ('saber', 0.16213643550872803),
 ('poco', 0.16049572825431824),
 ('sabrás', 0.15061169862747192),
 ('libro', 0.14246656000614166),
 ('aprendió', 0.13835258781909943),
 ('ha', 0.13711676001548767),
 ('color', 0.1354307234287262)]

### 5 - Visualizar agrupación de vectores

In [24]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [40]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



El modelo propuesto logra capturar similitudes entre palabras de canciones, aunque para lograr una mejor performance, el dataset deberia ser muchos mas grande. Esto mismo se se en el grafico de TSNE, donde haciendo zoon de determinadas regiones, hay cercania entre palabras de conocidas canciones.  